In [0]:
DROP SCHEMA if exists dev.control CASCADE;
DROP SCHEMA IF EXISTS dev.retail_batch_sql CASCADE;

In [0]:
CREATE CATALOG if not exists Dev;
CREATE SCHEMA Dev.Control;
CREATE SCHEMA Dev.retail_batch_sql;

### Controls

In [0]:
CREATE OR REPLACE TABLE Dev.Control.Control_Tables_Moviments (
  Project_Name string,
  Table_name string,
  Moviment_Date date,
  Last_Moviment_to_Run date,
  Offset_Days string,
  Running_Status string,
  StartDate timestamp,
  EndDate timestamp,
  Last_Execution_Duration float,
  Avarage_Time_Over_20_times float
) USING DELTA;

CREATE OR REPLACE TABLE Dev.Control.DEPENDENCIES (
  Table_name string,
  Dependens_on string) USING DELTA;

CREATE OR REPLACE VIEW dev.control.View_Dependencies_Dates AS
  SELECT
    A.Project_Name,
    A.Table_name,
    A.Moviment_Date,
    date_sub(current_date(), A.Offset_Days) AS Expected_Moviment_Date,
    c.project_name as Project_Name_of_Dependent_Table,
    b.Dependens_on,
    c.Moviment_Date as Moviment_Date_of_Dependent_Table
  FROM dev.control.Control_Tables_Moviments A
  INNER JOIN dev.control.dependencies B ON A.Table_name = B.Table_name 
  INNER JOIN dev.control.Control_Tables_Moviments C ON B.Dependens_on = C.Table_name
;

CREATE OR REPLACE VIEW dev.control.View_Dates AS
  SELECT
    Project_Name,
    Table_name,
    Moviment_Date,
    min(Moviment_Date_of_Dependent_Table) AS Able_To_Run_Day_of_Movement,
    TRANSFORM(
      SLICE( SEQUENCE((Moviment_Date), min(Moviment_Date_of_Dependent_Table)), 2, 1000 ),
      x -> DATE_FORMAT(x, 'yyyy-MM-dd')
    ) 
    AS List_of_Dates
  FROM dev.control.view_dependencies_dates
  GROUP BY Project_Name,Table_name, Moviment_Date;




In [0]:
INSERT INTO dev.control.Control_Tables_Moviments VALUES
('Retail', 'bronze_customers', '2024-12-31', '9999-12-31', 1, null, null, null, null, null),
('Retail', 'bronze_products', '2024-12-31', '9999-12-31', 1, null, null, null, null, null),
('Retail', 'bronze_sales', '2024-12-31', '9999-12-31', 1, null, null, null, null, null),
('Retail', 'silver_customers', '2024-12-31', '9999-12-31', 1, null, null, null, null, null),
('Retail', 'silver_products', '2024-12-31', '9999-12-31', 1, null, null, null, null, null),
('Retail', 'silver_sales', '2024-12-31', '9999-12-31', 1, null, null, null, null, null),
('Retail', 'gold_customers', '2024-12-31', '9999-12-31', 1, null, null, null, null, null),
('Retail', 'gold_products', '2024-12-31', '9999-12-31', 1, null, null, null, null, null),
('Retail', 'gold_sales', '2024-12-31', '9999-12-31', 1, null, null, null, null, null)

num_affected_rows,num_inserted_rows
9,9


In [0]:
INSERT INTO dev.control.DEPENDENCIES VALUES
('silver_customers', 'bronze_customers'),
('silver_products', 'bronze_products'),
('silver_sales', 'bronze_sales'),
('gold_customers', 'silver_customers'),
('gold_products', 'silver_products'),
('gold_sales', 'silver_sales'),
('gold_sales', 'gold_customers'),
('gold_sales', 'gold_products')


num_affected_rows,num_inserted_rows
8,8


In [0]:
USE Dev.retail_batch_sql

# Customers

In [0]:
CREATE OR REPLACE TABLE Bronze_Customers (
  Customer_ID bigint,
  Social_Number string,
  Name string,
  email string, 
  Address string, 
  ------------------------------- Operational fields:
  Registration_TimeStamp timestamp,
  Operation string,
  ------------------------------- DW fields:
  Moviment_Date date,
  Source_File string
) USING DELTA;

CREATE OR REPLACE TABLE Silver_Customers (
  Customer_ID bigint,
  Social_Number string,
  Name string,
  email string, 
  Address string, 
  ------------------------------- Operational fields:
  Registration_TimeStamp timestamp,
  Operation string,
  ------------------------------- DW fields:
  Moviment_Date date
) USING DELTA;

CREATE OR REPLACE TABLE Quarentine_Customers (
  Customer_ID bigint,
  Social_Number string,
  Name string,
  email string, 
  Address string, 
  ------------------------------- Operational fields:
  Registration_TimeStamp timestamp,
  Operation string,
  ------------------------------- DW fields:
  Moviment_Date date,
  Source_File string,
  Status string        --> Mark as valid or invalid
) USING DELTA;

CREATE OR REPLACE TABLE gold_customers (
  Customer_SK string,
  Customer_ID bigint,
  Social_Number string,
  Name string,
  email string, 
  Address string, 
  ------------------------------- Operational fields:
  -- Registration_TimeStamp timestamp,
  -- Operation string,
  ------------------------------- DW fields:
  Moviment_Date date,
  StartDate timestamp, --> Registration_TimeStamp 
  EndDate timestamp,
  hash string
) USING DELTA;

# Product

In [0]:
CREATE OR REPLACE TABLE Bronze_Products (
  Product_ID bigint,
  SKU string,
  Name string,
  Unit_Price float, 
  ------------------------------- Operational fields:
  Registration_TimeStamp timestamp,
  Operation string,
  ------------------------------- DW fields:
  Moviment_Date date,
  Source_File string
) USING DELTA;

CREATE OR REPLACE TABLE Silver_Products (
  Product_ID bigint,
  SKU string,
  Name string,
  Unit_Price float, 
  ------------------------------- Operational fields:
  Registration_TimeStamp timestamp,
  Operation string,
  ------------------------------- DW fields:
  Moviment_Date date
) USING DELTA;

CREATE OR REPLACE TABLE Quarentine_Products (
  Product_ID bigint,
  SKU string,
  Name string,
  Unit_Price float, 
  ------------------------------- Operational fields:
  Registration_TimeStamp timestamp,
  Operation string,
  ------------------------------- DW fields:
  Moviment_Date date,
  Source_File string,
  Status string        --> Mark as valid or invalid
) USING DELTA;

CREATE OR REPLACE TABLE gold_products (
  Product_SK string,
  Product_ID bigint,
  SKU string,
  Name string,
  Unit_Price float, 
  ------------------------------- DW fields:
  Moviment_Date date,
  StartDate timestamp, --> Registration_TimeStamp 
  EndDate timestamp,
  hash string
) USING DELTA;

### Sales

In [0]:
CREATE OR REPLACE TABLE Bronze_Sales (
  Sales_ID bigint,
  Customer_ID bigint,
  Product_ID bigint,
  Revenue float,
  Units_Sold int,
  ------------------------------- Operational fields:
  Registration_TimeStamp timestamp,
  Operation string,
  ------------------------------- DW fields:
  Moviment_Date date,
  Source_File string
) USING DELTA;

CREATE OR REPLACE TABLE Silver_Sales (
  Sales_ID bigint,
  Customer_ID bigint,
  Product_ID bigint,
  Revenue float,
  Units_Sold int,
  ------------------------------- Operational fields:
  Registration_TimeStamp timestamp,
  Operation string,
  ------------------------------- DW fields:
  Moviment_Date date
) USING DELTA;

CREATE OR REPLACE TABLE Quarentine_Sales (
  Sales_ID bigint,
  Customer_ID bigint,
  Product_ID bigint,
  Revenue float,
  Units_Sold int,
  ------------------------------- Operational fields:
  Registration_TimeStamp timestamp,
  Operation string,
  ------------------------------- DW fields:
  Moviment_Date date,
  Source_File string,
  Status string        --> Mark as valid or invalid
) USING DELTA;

CREATE OR REPLACE TABLE gold_sales (
  Sales_SK string,
  Sales_ID bigint,
  Customer_SK string,
  Product_SK string,
  Revenue float,
  Units_Sold int,
  ------------------------------- Operational fields:
  Sales_Timestamp timestamp,
  ------------------------------- DW fields:
  Moviment_Date date
) USING DELTA;

Executing subquery: DROP SCHEMA if exists dev.control CASCADE.
Executing subquery: DROP SCHEMA IF EXISTS dev.retail_batch_sql CASCADE.
Executing subquery: CREATE CATALOG if not exists Dev.
Executing subquery: CREATE SCHEMA Dev.Control.
Executing subquery: CREATE SCHEMA Dev.retail_batch_sql.
Executing subquery: CREATE OR REPLACE TABLE Dev.Control.Control_Tables_Moviments (
  Project_Name string,
  Table_name string,
  Moviment_Date date,
  Last_Moviment_to_Run date,
  Offset_Days string,
  Running_Status string,
  StartDate timestamp,
  EndDate timestamp,
  Last_Execution_Duration float,
  Avarage_Time_Over_20_times float
) USING DELTA.
Executing subquery: CREATE OR REPLACE VIEW dev.control.View_Dependencies_Dates AS
  SELECT
    A.Project_Name,
    A.Table_name,
    A.Moviment_Date,
    date_sub(current_date(), A.Offset_Days) AS Expected_Moviment_Date,
    c.project_name as Project_Name_of_Dependent_Table,
    b.Dependens_on,
    c.Moviment_Date as Moviment_Date_of_Dependent_Table
  FR